In [1]:
#로컬 파일 경로 
import os 
path=os.getcwd()
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/apple1/Projects/SoundCloud/attention.py
/Users/apple1/Projects/SoundCloud/new_data
/Users/apple1/Projects/SoundCloud/new_data.csv
/Users/apple1/Projects/SoundCloud/summarization.ipynb
/Users/apple1/Projects/SoundCloud/classification.ipynb
/Users/apple1/Projects/SoundCloud/new_data_60000
/Users/apple1/Projects/SoundCloud/vcrawling.ipynb


In [2]:
import pandas as pd 
df_y = pd.read_pickle("/Users/apple1/Projects/SoundCloud/new_data_60000")

In [3]:
df_y.head()

,overall,reviewText,summary,pretreatment
0,5.0,this is a great cd full of worship favorites!!...,great worship cd,great cd full worship favorite time great ke...
1,5.0,"so creative! love his music - the words, the ...",gotta listen to this!,creative love music word message favorite ...
2,5.0,"keith green, gone far to early in his carreer,...",great approach still gets the message out,keith green gone far early carreer left u go...
3,5.0,keith green had his special comedy style of ch...,great a must have,keith green special comedy style chirstian mus...
4,5.0,keith green / so you wanna go back to egypt......,a great one from keith with a guest appearance...,keith green wan na go back egypt : album kei...


In [5]:
#overall 1,2 -> 0(부정)/ 4,5->1(긍정)으로 변환해야 함. 
df2=df_y.replace({'overall': {1.0: 0, 2.0: 0, 4.0: 1, 5.0: 1}}).copy()

ValueError: Replacement not allowed with overlapping keys and values

In [ ]:
# 부정 긍정 라벨 몇개인지 확인. 밸런스 안맞으면 데이터 잘라서 맞추기  코드 추가하기 
import matplotlib.pyplot as plt
stars=df2_y['sentiment']
print(sorted(stars.unique())) #고유값 나열
plt.hist(stars, bins=2, rwidth=0.85)
plt.title('Distribution of label')
plt.xlabel('label')
plt.ylabel('number')

In [ ]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
texts_y=[]
for line in df_y['retreatment']:
    texts_y.append(line)

In [ ]:
print(len(texts_y),texts_y[0])

In [ ]:
from tensorflow.python.keras.datasets import reuters
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.models import load_model

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(text_y)

threshold = 7
total_cnt = len(src_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


In [7]:
import numpy as np
text_len = [len(s.split()) for s in df_y['pretreatment']]

#print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
#print('요약의 최소 길이 : {}'.format(np.min(summary_len)))
print('요약의 최대 길이 : {}'.format(np.max(summary_len)))
print('요약의 평균 길이 : {}'.format(np.mean(summary_len)))

텍스트의 최대 길이 : 991
텍스트의 평균 길이 : 26.23775


NameError: name 'summary_len' is not defined

In [ ]:
src_vocab=1230
maxlen2=50
t2=Tokenizer(num_words = src_vocab)
t2.fit_on_texts(texts_y)
sequences2=t2.texts_to_sequences(texts_y)
texts_pad=sequence.pad_sequences(sequences2,maxlen=maxlen2,padding='post',truncating='post')

# lstm 모델1

In [ ]:
#train data, test data 분리
x_train, x_test, y_train, y_test = train_test_split(texts_pad, to_categorical(df2_y['sentiment']), test_size = 0.1, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=1)

In [ ]:
y_model = Sequential()
y_model.add(Embedding(4300, 120))
y_model.add(LSTM(120))
y_model.add(Dense(2, activation='softmax'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('you2_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
y_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
history4 = y_model.fit(x_train, y_train, batch_size=128, epochs=100, callbacks=[es, mc], validation_data=(x_val, y_val))


In [ ]:
loaded_model = load_model('you2_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

In [ ]:
accu_plot(history4)

# lstm 모델 2

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold=StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
cvscores=[]
X=texts_pad
Y=df2_y['sentiment']
num=0
index_right=[]
index_wrong=[]


        for train, test in kfold.split(X,Y):
            i_r=[]
            i_w=[]
            y_model = Sequential()
            y_model.add(Embedding(4300, 100))
            #수정
            y_model.add(LSTM(100, return_sequences=True, recurrent_dropout=0.4))
            y_model.add(LSTM(100, return_sequences=True))
            y_model.add(Dropout(0.5))
            y_model.add(LSTM(100))
            y_model.add(Dense(2, activation='softmax'))
            y_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
            history3 = y_model.fit(X[train], to_categorical(Y[train]), batch_size=128, epochs=30, verbose=0)
            scores=y_model.evaluate(X[test], to_categorical(Y[test]),verbose=0)
            print("%s: %.2f%%" % (y_model.metrics_names[1], scores[1]*100))
            cvscores.append(scores[1] * 100)
            y_model.save_weights('checkpoints_c/lstm3/my_checkpoint'+str(num))


            ynew = y_model.predict_classes(X[test])
            file_name='predict_count/lstm3/predict_wrong'+str(num)+'.txt'
            f = open(file_name, "w")
            file_name2='predict_count/lstm3/predict_right'+str(num)+'.txt'
            f2 = open(file_name2, "w")
            for i, real in enumerate(Y[test]):
        
                line=''
                for integer in X[test][i]:
                    for word, index in t2.word_index.items():
                        if index == integer:
                            line+=(' '+word)
                    #print(line, y_test[i], ynew[i])
                if(real!=ynew[i]):
                    f.write("index: %s, real:%s, predict:%s\n%s\n" % (str(i),str(real),str(ynew[i]),line))
                    i_w.append(i)
                else: 
                    f2.write("index: %s, real:%s, predict:%s\n%s\n" % (str(i),str(real),str(ynew[i]),line))
                    i_r.append(i)
            index_wrong.append(i_w)
            index_right.append(i_r)
                
    
            f.close
            f2.close
            num+=1 
        print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



In [ ]:
y_model.summary()